## Mini Chess Solver Using Dynamic Programming - Total 7 Marks

### Problem Statement

Design and implement a reinforcement learning agent using dynamic programming (value iteration or policy iteration) to compute an optimal policy for a simplified chess game. The agent plays as White and must learn how to convert an advantage into a win or at least avoid a loss in a MiniChess game against a defensive opponent. The problem must be modelled as a finite MDP. Register number of first student in a group (alphabetically sorted) will be considered for configuration design.
The student will:
* Implement a custom Mini Chess environment.
* Use dynamic programming to compute the optimal value function and policy.
* Analyze how state design and reward shaping affect the learned policy and convergence.

### Scenario

You are building a “Mini Chess Game” for beginner players. The coach focuses on a small, tractable game:
* White: King + Pawn
* Black: King
* Board: 4×4 or 5×5 MiniChess board
* White moves first and tries to either:
    * Promote the pawn and then deliver checkmate, or
    * Force a checkmate directly (if possible)

Black tries to prevent this by blocking the pawn, chasing the white king, or capturing the pawn. The game is restricted to this small set of pieces and a tiny board.

### Environment Description
#### Board and Pieces
* Board size:
    * If the student roll number / registration number is even: use a 4×4 board (rows 0–3, cols 0–3).
    * If odd: use a 5×5 board (rows 0–4, cols 0–4).
* Pieces always present:
    * White King (WK)
    * White Pawn (WP)
    * Black King (BK)
    * No castling, no en passant, no promotion to anything other than Queen.
* Legal Moves
    * Kings move like normal chess kings - one square in any direction (8- neighborhood), staying on the board.
    * Pawn:
        * Moves one square forward (towards larger row index or smaller row index – the student must choose and clearly document a convention).
        * Captures diagonally forward by one square.
    * All usual constraints apply:
        * Kings cannot move into check.
        * Two kings may never occupy adjacent squares (illegal state).
        * A piece cannot move through other pieces.
* Episode Termination
    * An episode ends when any of the following happens:
        * Checkmate (White checkmates Black).
        * Stalemate (side to move has no legal moves but is not in check).
        * Pawn Capture (Black captures the White pawn).
        * Pawn Promotion (White pawn reaches last rank and becomes a Queen). After promotion, they may either:
                * (a) terminate immediately with a reward, or
                * (b) continue playing with a Queen replacing the pawn.
        * The student must choose one approach and justify it.
        * Move limit exceeded (e.g., 20 or 30 plies) – draw
1. State Space

* Each state should minimally encode:
    * Coordinates of WK: (r_wk, c_wk)
    * Coordinates of WP (or a special value if promoted/captured): (r_wp, c_wp) or status flag
    * Coordinates of BK: (r_bk, c_bk)
    * Player to move: {White, Black}
    * Any additional flags that can be necessary like,
        * Has the pawn promoted?
        * Check / checkmate / stalemate indicators.
* The student must:
    * Describe the state representation clearly.
2. Action Space
    * For each state, actions are the legal moves for the side to move:
        * Move King to a legal square
        * Move Pawn / promoted Queen
    * The student must implement a function that, given a state, returns all legal actions.
3. Rewards
* The student has to define the reward schemes like:
    * White checkmates Black: +10
    * Pawn gets captured: -10
    * Stalemate or draw by move limit: 0
    * All non-terminal moves: 0

In [ ]:
# Import Statements
import itertools
import math
import time
from collections import deque, defaultdict
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional, Iterable, Any


import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Basic types and helpers

Pos = Tuple[int, int] # (row, col)


@dataclass(frozen=True)
class State:
    # define wk,wp,bk and the other things needed
    wk: Pos
    wp: Optional[Pos] # None if captured or promoted
    bk: Pos
    to_move: str # 'W' or 'B'
    promoted: bool = False

    def as_tuple(self):
        return (self.wk, self.wp, self.bk, self.to_move, self.promoted)

@dataclass(frozen=True)
class Action:
    piece: str # 'K' or 'P' or 'Q' (after promotion)
    src: Pos
    dst: Pos

    def as_tuple(self):
        return (self.piece, self.src, self.dst)


In [ ]:
# Define the MiniChess Environment - 1.5 mark
class MiniChessEnv:

    def __init__():

        pass



    # Define functions for board & positions

    def on_board():

        pass

    def king_moves():

        pass

    def pawn_moves():

        pass

    # Define functions for game rules and legality checks

    def kings_adjacent():

        pass

    def in_check():

        pass

    def legal_actions(): # Return a list of legal actions for the side to move in the given state

        # This function must enforce:
        # Kings cannot move into squares adjacent to the other king
        # Kings cannot move onto squares occupied by their own piece
        # Pawn cannot move onto a square occupied by same-color piece
        # Pawn captures are allowed (if BK on diagonal)

        pass

    # Transition / step

    def step():
        # Apply action to state, return (next_state, reward, done, info)
        # Reward scheme is :
        # checkmate: +10
        # pawn captured by black: -10
        # draw / stalemate : 0

        # Implement checkmate and stalemate detection


        pass

    def reset():

        pass

    def render():
        # print to move and promoted

        pass


In [ ]:
# State encoding & Listing - 1 mark

class StateIndexer:

    def __init__():

        pass

    def _build():

        pass

    def encode():

        pass

    def decode():

        pass

# List all reachable states (BFS) from an initial state - 1 mark

def list_reachable():

    pass


In [ ]:
# Value Iteration / Policy Iteration - 1 mark

def value_iteration():

    pass

# or

def policy_iteration():


    pass


In [ ]:
# Visualization - 0.5 mark

def plot_value():

    # Fix pawn and black king positions; vary white king position and show heatmap of V(s).

In [ ]:
# Main Usage - 1 mark

if __name__ == '__main__':
    env=MiniChessEnv()

    # Example initial state (students must define based on ID rule)

    # List reachable states

    # Run value or policy iteration on reachable states

    # Visualize

    # Save and print policy for a few states

In [ ]:
# Result Discussion and descriptive answers, conclusion - 1 mark